In [1]:
from Environment import Policy, rewardFunc, Environment
import numpy as np

from Generate_Dataset import Generate_Dataset
from torch.utils.data import DataLoader, TensorDataset, random_split
from torch.nn import CrossEntropyLoss
import torch
import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
DATASET_SIZE = 10000
INPUT_LOWER_LIMIT = 1
INPUT_UPPER_LIMIT = 100

NUM_EPOCHS = 1000

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
dataset_obj = Generate_Dataset("../LLM_Critic/pretrainedCritic.pt", DATASET_SIZE, INPUT_LOWER_LIMIT, INPUT_UPPER_LIMIT, device)
dataset = dataset_obj.getX().to(device).to(torch.float32)
model = Policy(1,64,1000).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = Environment()

/home/hvgupta/fincatch/AI_Models/Actor/Generate_Dataset.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(CriticPath)


In [ ]:
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

gamma = 0.8
for epoch in range(NUM_EPOCHS):
    model.eval()
    G = []
    Action = []
    Reward = []
    for i, data in enumerate(dataloader):
        data.requires_grad_(True)
        output = model(data)
        action = torch.argmax(output, dim=1)
        reward,_ = loss_fn(data.view(1,-1), action.view(-1,1), device)
        G.append(gamma**i * reward)
        Action.append(action)
        Reward.append(reward)
    
    G = torch.tensor(G).flip(dims=(0,))
    G = (G - G.mean()) / (G.std() + 1e-9)
    loss = 0
    for g, action in zip(G, Action):
        loss += -g * torch.log(output[action])
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(f"Epoch: {epoch}, Loss: {loss.item()}")

RuntimeError: a Tensor with 32 elements cannot be converted to Scalar